In [141]:
using LinearAlgebra
using SparseArrays

In [142]:
# Variables Globales

mi = 10; nj = 12

row_numbers = (mi - 2) * (nj - 2)
column_numbers = (mi - 2) * (nj - 2)
A = zeros(row_numbers, column_numbers)
b = zeros(column_numbers)

# Define other parameters
delta_x = 1.0 / (mi - 1)
delta_y = 1.0 / (nj - 1)
cond_ter = 1.5 # 100.0
temp_ini = 3.8 # 308.0
temp_fin = 2.8 # 298.0
flux_aba = 3.8 # 308.0
flux_arr = 3.8 # 308.0

# Initialize matrices with zeros
BI = zeros((nj, mi))
AI = zeros((mi, nj))
AC = zeros((mi, nj))
AD = zeros((mi, nj))
BD = zeros((nj, mi))

12×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [143]:
function build_global_matrix(A)
    tope = (mi -2) * (nj - 2)
    modulo = nj - 2
    for ii in range(1, tope)
        A[ii, ii] = 2.0 * cond_ter * (1.0 / (delta_x * delta_x) + 1.0 / (delta_y * delta_y))
    end
    # BI - S
    for ii in range(nj - 1, tope)
        A[ii, ii - nj + 2] = -1.0 * cond_ter / (delta_y * delta_y)
    end
    # AI
    for ii in range(2, tope)
        if (ii - 1) % modulo == 0
            # A[ii, ii - 1] = 88
            continue
        end
        A[ii, ii - 1] = -1.0 * cond_ter / (delta_x * delta_x)
    end
    # AD
    for ii in range(1, tope - 1)
        if ii % modulo == 0
            continue
        end
        A[ii, ii + 1] = -1.0 * cond_ter / (delta_x * delta_x)
    end
    # BD - N
    for ii in range(1, tope - nj + 2)
        A[ii, ii + nj - 2] = -1.0 * cond_ter / (delta_y * delta_y)
    end
end

build_global_matrix (generic function with 1 method)

In [144]:
using Printf
build_global_matrix(A)
tope = (mi -2) * (nj - 2)

for i in range(1, tope)
    for j in range(1, tope)
        s = @sprintf("%6.1f ", A[i, j])
        print(s)
    end
    println()
end

 606.0 -121.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0 -181.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0 
-121.5  606.0 -121.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0 -181.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.

In [145]:
function ensambla_tdmax(AI, AC, AD, deltax, deltay, cond_ter, temp_ini, temp_fin, jj)

    # Define boundary conditions
    AI[1, jj] = -temp_ini * cond_ter / (deltax * deltax)
    AD[mi, jj] = -temp_fin * cond_ter / (deltax * deltax)

    # Assemble the tridiagonal matrix and the result vector
    for iin in range(2, mi - 1)
        AI[iin, jj] = -1.0 * cond_ter / (deltax * deltax)
        AC[iin, jj] = 2.0 * cond_ter * (1.0 / (deltax * deltax) + 1.0 / (deltay * deltay))
        AD[iin, jj] = -1.0 * cond_ter / (deltax * deltax)
    end
end

function ensambla_tdmay(BI, BD, deltax, deltay, cond_ter, flux_aba, flux_arr, ii)

    # Define boundary conditions
    BI[1, ii] = -flux_aba * cond_ter / (deltay * deltay)
    BD[nj, ii] = -flux_arr * cond_ter / (deltay * deltay)

    # Assemble the tridiagonal matrix and the result vector
    for jjn in range(2, nj - 1)  # Loop from 1 to nj-2
        BI[jjn, ii] = -1.0 * cond_ter / (deltay * deltay)
        BD[jjn, ii] = -1.0 * cond_ter / (deltay * deltay)
    end
end

ensambla_tdmay (generic function with 1 method)

In [146]:
for jj in range(2, nj - 1)
    ensambla_tdmax(AI, AC, AD, delta_x, delta_y, cond_ter, temp_ini, temp_fin, jj)
end
for ii in range(2, mi - 1)
    ensambla_tdmay(BI, BD, delta_x, delta_y, cond_ter, flux_aba, flux_arr, ii)
end

In [147]:
function get_vector_independent_terms(BI, AI, AD, BD, b, nj, mi)
    kk = 1
    for jj in range(2, nj - 1)
        for ii in range(2, mi - 1)
            if jj == 2
                b[kk] += -BI[jj - 1, ii]
            end

            if ii == 2
                b[kk] += -AI[ii - 1, jj]
            end

            if ii == mi - 1
                b[kk] += -AD[ii + 1, jj]
            end

            if jj == nj - 1
                b[kk] += -BD[jj + 1, ii]
            end

            kk += 1
        end
    end
end

get_vector_independent_terms (generic function with 1 method)

In [149]:
b = zeros(column_numbers)
get_vector_independent_terms(BI, AI, AD, BD, b, nj, mi)
for i in range(1, column_numbers)
    s = @sprintf("%6.1f ", b[i])
    print(s)
    println()
end

1151.4 
 689.7 
 689.7 
 689.7 
 689.7 
 689.7 
 689.7 
1029.9 
 461.7 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
 340.2 
 461.7 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
 340.2 
 461.7 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
 340.2 
 461.7 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
 340.2 
 461.7 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
 340.2 
 461.7 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
 340.2 
 461.7 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
 340.2 
 461.7 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
   0.0 
 340.2 
1151.4 
 689.7 
 689.7 
 689.7 
 689.7 
 689.7 
 689.7 
1029.9 
